In [ ]:
#import necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set()

In [ ]:
#import the read the dataset
heart= pd.read_csv('../input/heart-disease/heart.csv')
heart.head()


Attribute Information:

        Age: Age
        Sex: Sex (1 = male; 0 = female)
        ChestPain: Chest pain (typical, asymptotic, nonanginal, nontypical)
        RestBP: Resting blood pressure
        Chol: Serum cholestoral in mg/dl
        Fbs: Fasting blood sugar > 120 mg/dl (1 = true; 0 = false)
        RestECG: Resting electrocardiographic results
        MaxHR: Maximum heart rate achieved
        ExAng: Exercise induced angina (1 = yes; 0 = no)
        Oldpeak: ST depression induced by exercise relative to rest
        Slope: Slope of the peak exercise ST segment
        Ca: Number of major vessels colored by flourosopy (0 - 3)
        Thal: (3 = normal; 6 = fixed defect; 7 = reversable defect)
        target: AHD - Diagnosis of heart disease (1 = yes; 0 = no)


#### Understanding the Dataset

In [ ]:
#shape of the dataset
heart.shape

In [ ]:
heart.columns

In [ ]:
#summary of the dataset
heart.info()

In [ ]:
#Checking missing value in dataset
heart.isnull().sum()

In [ ]:
#Check for unique values in columns
heart.nunique()

In [ ]:
#statistical summary of the dataset
heart.describe()

In [ ]:
heart['target'].value_counts()

We will almost balanced target class

##### Initial Observation
    There are 303 records and 14 columns are present
    Mixture of Integer and flaot datatypes datas are present. basically Dataset consisits numerical columns
    Surprisingly, There are no missing values present in any of the columns
    When we looked at unique values in each columns, barring trestbps, chol, thalach, oldpeak all remaining columns needs to be converted to categorical type
    We need to predit presence (value 1) or absence (value 0) of heart disease in the patient which are given in target column.
    Since we know the target class, this is supervised learning, typical binary Classification modelling.

In [ ]:
#convert columns to category
columns_to_category= ['sex', 'cp', 'fbs', 'restecg','exang', 'slope', 'ca', 'thal', 'target']

heart[columns_to_category]= heart[columns_to_category].astype('category')

In [ ]:
heart.describe()

#### Observation
    - mean and median of age, trestbps, oldpeaks are almost similar. Outliers may have less impact
    - mean is higher than median which tells outliers might have present in both the columns. Data Distribution is skewed towards right

### Exploratory Data Analytics

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

sns.kdeplot(heart[heart["target"]==1]["age"], shade=True, color="blue", label="Presence of Heart Disease", ax=ax)
sns.kdeplot(heart[heart["target"]==0]["age"], shade=True, color="green", label="Absence of Heart Disease", ax=ax)

fig.suptitle("Presence of Heart Disease by Age")

ax.legend();

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

sns.kdeplot(heart[heart["target"]==1]["trestbps"], shade=True, color="blue", label="Presence of Heart Disease", ax=ax)
sns.kdeplot(heart[heart["target"]==0]["trestbps"], shade=True, color="green", label="Absence of Heart Disease", ax=ax)

fig.suptitle("Presence of Heart Disease by Blood Sugar")

ax.legend();

There is no significant pattern found in Target against Blood Sugar. Blood Sugar might not be good predictor of target.

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

sns.kdeplot(heart[heart["target"]==1]["chol"], shade=True, color="blue", label="Presence of Heart Disease", ax=ax)
sns.kdeplot(heart[heart["target"]==0]["chol"], shade=True, color="green", label="Absence of Heart Disease", ax=ax)

fig.suptitle("Presence of Heart Disease based on Cholestrol")

ax.legend();

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

sns.kdeplot(heart[heart["target"]==1]["thalach"], shade=True, color="blue", label="Presence of Heart Disease", ax=ax)
sns.kdeplot(heart[heart["target"]==0]["thalach"], shade=True, color="green", label="Absence of Heart Disease", ax=ax)

fig.suptitle("Presence of Heart Disease based on thalach")

ax.legend();

Based on kde plots, it looks like age, Thalach are good predictor for outcome variable.

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

sns.kdeplot(heart[heart["target"]==1]["oldpeak"], shade=True, color="blue", label="Presence of Heart Disease", ax=ax)
sns.kdeplot(heart[heart["target"]==0]["oldpeak"], shade=True, color="green", label="Absence of Heart Disease", ax=ax)

fig.suptitle("Presence of Heart Disease based on OldPeak")

ax.legend();

In [ ]:
# Create the correlation matrix
corr = heart.corr()
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.figure(figsize=(14,10))
# Add the mask to the heatmap
sns.heatmap(corr, mask=mask, cmap='YlGnBu',center=0, linewidths=1, annot=True, fmt=".2f")
plt.show()

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(10,20))
sns.boxplot(x='target', y='age', data=heart, ax=ax[0])
ax[0].set_xlabel('Target', fontsize=12)

sns.boxplot(x='target', y='trestbps', data=heart, ax=ax[1])
ax[1].set_xlabel('Target', fontsize=12)

sns.boxplot(x='target', y='chol', data=heart, ax=ax[2])
ax[2].set_xlabel('Target', fontsize=12)

sns.boxplot(x='target', y='thalach', data=heart, ax=ax[3])
ax[3].set_xlabel('Target', fontsize=12)

sns.boxplot(x='target', y='oldpeak', data=heart, ax=ax[4])
ax[4].set_xlabel('Target', fontsize=12)

plt.show()

Notice more outliers are present in Oldpeak attribute. for now, we will do model building including considering outlier values

In [ ]:
fig = plt.figure(figsize=(15,10))
sns.pairplot(data=heart, vars=['age', 'trestbps', 'chol', 'thalach', 'oldpeak'], hue='target')
plt.subplots_adjust(top=0.9)
plt.show();

In [ ]:
heart.var()

Few of the features have high variance are in different scales. We need to center these variances around 0

In [ ]:
#split independent and dependent variable
X= heart.iloc[:, :13]
y= heart.iloc[:,-1]

#### Model Building

In [ ]:
#import necessary scikit learn libraries
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
category_cols= X.columns[X.dtypes == 'category']

encoded= pd.get_dummies(X[category_cols], drop_first=True)
encoded.head()

In [ ]:
concat_X= pd.concat([X,encoded], axis=1)
concat_X.shape

In [ ]:
final_X= concat_X.drop(['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thalach'], axis=1)
final_X.shape

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(final_X, y, random_state=99)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
scaler = MinMaxScaler()

lr = LogisticRegression(solver='liblinear', random_state=99) # Other solvers have failure to converge problem

pipeline = Pipeline([('scale',scaler), ('lr', lr),])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [ ]:
print ('Training accuracy: %.4f' % pipeline.score(X_train, y_train))

In [ ]:
print ('Test accuracy: %.4f' % pipeline.score(X_test, y_test))

In [ ]:
print ('Training accuracy score: %.4f' % accuracy_score(y_test, y_pred))

In [ ]:
# Confusion matrix
pd.DataFrame(confusion_matrix(y_test, y_pred))

This forms the basis of evaluating and modifying our models in the next section.

### Model Validation

Several things will be done:

    Cross validation with shuffle split
    Feature selections
    Grid search for hyperparameters
    Identify errors

In [ ]:
# Cross validation
from sklearn.metrics import make_scorer

scorer= make_scorer(accuracy_score)

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=99)
cv_score = cross_val_score(lr, X_train, y_train, cv=cv, scoring=scorer)
print('Cross validation accuracy score: %.3f' %np.mean(cv_score))

In [ ]:
## Feature selection
from sklearn.feature_selection import RFECV

steps = 20
n_features = len(X_train.columns)
X_range = np.arange(n_features - (int(n_features/steps)) * steps, n_features+1, steps)

rfecv = RFECV(estimator=lr, step=steps, cv=cv, scoring=scorer)

pipeline2 = Pipeline([('scale',scaler), ('rfecv', rfecv)])
pipeline2.fit(X_train, y_train)
plt.figure(figsize=(10,6))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(np.insert(X_range, 0, 1), rfecv.grid_scores_)
plt.show()

In [ ]:
print ('Optimal no. of features: %d' % np.insert(X_range, 0, 1)[np.argmax(rfecv.grid_scores_)])

Features used to build the model are optimal features

In [ ]:
from sklearn.model_selection import GridSearchCV

grid={"C":np.logspace(-2,2,5), "penalty":["l1","l2"]}
searcher_cv = GridSearchCV(lr, grid, cv=cv, scoring = scorer)
searcher_cv.fit(X_train, y_train)

print("Best parameter: ", searcher_cv.best_params_)
print("accuracy score: %.3f" %searcher_cv.best_score_)

Default Parameter helps model perform Better

### Model Evaluation

Accuracy is a good measure here because data is balanced. we can also use AUC_BOC score as evaluation metric 

In [ ]:
from sklearn.metrics import roc_curve, auc

#compute predicted probabilities: y_pred_prob
y_pred_prob= pipeline.predict_proba(X_test)[:,1]


#Generate ROC curve values: fpr, tpr, thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Calculate the AUC

roc_auc = auc(fpr, tpr)
print ('ROC AUC: %0.3f' % roc_auc )

#Plot ROC curve
plt.figure(figsize=(10,8))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

The AUC for both the test and train samples when run on my logistic regression demonstrates relatively strong power of separation between positive and negative occurences (Presence of Heart Disease - 1, Absence of Heart Disease - 0)

### Model Comparison

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

nb= MultinomialNB()
dt= DecisionTreeClassifier()
rf= RandomForestClassifier(n_estimators=100)
gbr= GradientBoostingClassifier()
svm= SVC(kernel='linear')

In [ ]:
classifiers = [('Naive Bayes', nb),('Decision Tree Classifier', dt), ('RandomForest Classifier', rf), ('Gradient Boost', gbr), ('SVC', svm)]

# Iterate over the pre-defined list of regressors
for classifier_name, classifier in classifiers:   
    # Fit clf to the training set
    classifier.fit(X_train, y_train)    
    y_pred = classifier.predict(X_test) 
    
    training_set_score = classifier.score(X_train, y_train)
    test_set_score = classifier.score(X_test, y_test)
    
    
    print('{:s} : {:.3f}'.format(classifier_name, training_set_score))
    print('{:s} : {:.3f}'.format(classifier_name, test_set_score))

Looks like SVM model will perform better than other models in this dataset